# 4편: GAN으로 이미지 생성하기

이 노트북에서는 다음을 학습합니다:
- 분류 모델 vs 생성 모델의 차이
- GAN(Generative Adversarial Network)의 구조와 원리
- Generator와 Discriminator 구현
- FashionMNIST 이미지를 생성하는 GAN 학습

## 1. 생성 모델이란?

지금까지 학습한 모델은 **분류(Discriminative) 모델**입니다:
- 입력: 이미지 → 출력: 클래스 레이블
- "이 이미지가 무엇인지" 판별

**생성(Generative) 모델**은 반대입니다:
- 입력: 랜덤 노이즈 → 출력: 새로운 이미지
- "학습 데이터와 비슷한 새로운 데이터" 생성
- 데이터의 **확률 분포**를 학습

## 2. GAN의 구조

GAN은 두 개의 신경망이 서로 **경쟁(적대적 학습)**하며 발전합니다:

- **Generator(생성자)**: 랜덤 노이즈로부터 가짜 이미지를 생성하는 "위조범"
- **Discriminator(판별자)**: 진짜/가짜 이미지를 구별하는 "감정사"

학습이 진행되면:
- Generator는 점점 정교한 가짜 이미지를 만들고
- Discriminator는 점점 정밀하게 판별하려 하고
- 최종적으로 Generator가 진짜와 구별 불가능한 이미지를 생성

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

In [ ]:
# 데이터 준비 (정규화: [0,1] → [-1,1])
standardizer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train_data = dsets.FashionMNIST(root="data/", train=True, transform=standardizer, download=True)
test_data = dsets.FashionMNIST(root="data/", train=False, transform=standardizer, download=True)

batch_size = 200
train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)

print(f"학습 데이터: {len(train_data)}장, 배치 수: {len(train_loader)}")

In [ ]:
# 이미지 시각화 헬퍼
def show_image(img):
    """단일 이미지 출력 (정규화 복원)"""
    img = (img + 1) / 2  # [-1,1] → [0,1]
    img = img.squeeze()
    plt.imshow(img.numpy(), cmap="gray")
    plt.axis("off")
    plt.show()

def show_grid(img):
    """이미지 그리드 출력"""
    img = utils.make_grid(img.cpu().detach())
    img = (img + 1) / 2
    npimg = img.numpy()
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()

## 3. 모델 구현

### Generator (생성자)
```
랜덤 노이즈 (100) → Linear(256) → ReLU → Linear(256) → ReLU → Linear(784) → Tanh → 이미지 (28×28)
```

### Discriminator (판별자)
```
이미지 (784) → Linear(256) → LeakyReLU → Linear(256) → LeakyReLU → Linear(1) → Sigmoid → 확률 [0,1]
```

In [ ]:
d_noise = 100   # 노이즈 차원
d_hidden = 256  # 은닉층 차원

def sample_noise(batch_size=1, d_noise=100):
    """랜덤 노이즈 생성"""
    return torch.randn(batch_size, d_noise, device=device)

# Generator: 노이즈 → 이미지
G = nn.Sequential(
    nn.Linear(d_noise, d_hidden),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, d_hidden),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, 28 * 28),
    nn.Tanh(),  # 출력 범위를 [-1, 1]로 제한
).to(device)

# Discriminator: 이미지 → 진짜일 확률
D = nn.Sequential(
    nn.Linear(28 * 28, d_hidden),
    nn.LeakyReLU(0.2),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, d_hidden),
    nn.LeakyReLU(0.2),
    nn.Dropout(0.1),
    nn.Linear(d_hidden, 1),
    nn.Sigmoid(),  # 출력: 진짜일 확률 [0, 1]
).to(device)

print("=== Generator ===")
print(f"파라미터 수: {sum(p.numel() for p in G.parameters()):,}")
print(f"\n=== Discriminator ===")
print(f"파라미터 수: {sum(p.numel() for p in D.parameters()):,}")

In [ ]:
# 학습 전 Generator의 출력 확인 (완전한 노이즈)
z = sample_noise()
fake_img = G(z).view(28, 28)
print("학습 전 Generator 출력 (랜덤 노이즈에서 생성):")
show_image(fake_img.cpu().detach())

# Discriminator의 판별 결과
z = sample_noise(5)
fake_batch = G(z)
probs = D(fake_batch)
print(f"Discriminator 판별 결과 (학습 전): {probs.data.squeeze().tolist()}")
print(f"  → 약 0.5 (진짜/가짜 구별 못함)")

## 4. GAN 학습

매 배치마다 두 단계로 학습합니다:

**Discriminator 학습:**
- 진짜 이미지 → D → 1에 가까워지도록 (진짜를 진짜로 판별)
- 가짜 이미지 → D → 0에 가까워지도록 (가짜를 가짜로 판별)

**Generator 학습:**
- 가짜 이미지 → D → 1에 가까워지도록 (가짜를 진짜처럼 속이기)

In [ ]:
def train_one_epoch(generator, discriminator, opt_g, opt_d):
    """1 에포크 학습"""
    generator.train()
    discriminator.train()

    for img_batch, _ in train_loader:
        img_batch = img_batch.to(device)

        # --- Discriminator 학습 ---
        opt_d.zero_grad()
        # 진짜 이미지에 대한 판별
        p_real = discriminator(img_batch.view(-1, 28 * 28))
        # 가짜 이미지에 대한 판별
        p_fake = discriminator(generator(sample_noise(batch_size, d_noise)))

        # 진짜는 1, 가짜는 0으로 판별하도록 학습
        loss_real = -torch.log(p_real).mean()
        loss_fake = -torch.log(1.0 - p_fake).mean()
        loss_d = loss_real + loss_fake

        loss_d.backward()
        opt_d.step()

        # --- Generator 학습 ---
        opt_g.zero_grad()
        # 가짜 이미지를 진짜처럼 속이도록 학습
        p_fake = discriminator(generator(sample_noise(batch_size, d_noise)))
        loss_g = -torch.log(p_fake).mean()

        loss_g.backward()
        opt_g.step()


def evaluate(generator, discriminator):
    """테스트 데이터로 평가"""
    p_real, p_fake = 0.0, 0.0
    generator.eval()
    discriminator.eval()

    with torch.no_grad():
        for img_batch, _ in test_loader:
            img_batch = img_batch.to(device)
            p_real += torch.sum(discriminator(img_batch.view(-1, 28 * 28))).item() / len(test_data)
            p_fake += torch.sum(discriminator(generator(sample_noise(batch_size, d_noise)))).item() / len(test_data)

    return p_real, p_fake

In [ ]:
# 가중치 초기화
def init_weights(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_normal_(p)
        else:
            nn.init.uniform_(p, 0.1, 0.2)

init_weights(G)
init_weights(D)

opt_g = optim.Adam(G.parameters(), lr=0.0002)
opt_d = optim.Adam(D.parameters(), lr=0.0002)

# 학습 기록
p_real_history = []
p_fake_history = []

total_epochs = 100
print(f"GAN 학습 시작 ({total_epochs} 에포크)...\n")

for epoch in range(total_epochs):
    train_one_epoch(G, D, opt_g, opt_d)
    p_real, p_fake = evaluate(G, D)

    p_real_history.append(p_real)
    p_fake_history.append(p_fake)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{total_epochs} | p_real: {p_real:.4f} | p_fake: {p_fake:.4f}")
        show_grid(G(sample_noise(16)).view(-1, 1, 28, 28))

## 5. 학습 결과 분석

In [ ]:
# p_real / p_fake 수렴 과정
plt.figure(figsize=(10, 5))
plt.plot(p_real_history, label="p_real (진짜를 진짜로 판별)")
plt.plot(p_fake_history, label="p_fake (가짜를 진짜로 판별)")
plt.axhline(y=0.5, color="gray", linestyle="--", alpha=0.5, label="이상적 수렴점 (0.5)")
plt.xlabel("Epoch")
plt.ylabel("Probability")
plt.title("GAN Training Progress")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("p_real과 p_fake가 모두 0.5에 수렴하면 이상적인 상태입니다.")
print("→ Discriminator가 진짜/가짜를 구별하지 못한다는 의미")

In [ ]:
# 최종 생성 이미지
print("최종 Generator가 생성한 패션 아이템 이미지 (4×4):")
show_grid(G(sample_noise(16)).view(-1, 1, 28, 28))

## 정리

### GAN 핵심 개념

| 구성 요소 | 역할 | 목표 |
|---|---|---|
| Generator | 노이즈 → 가짜 이미지 | Discriminator를 속이기 |
| Discriminator | 이미지 → 진짜 확률 | Generator를 간파하기 |
| 적대적 학습 | 두 모델의 경쟁 | 균형점(Nash Equilibrium)에 도달 |

### GAN 학습의 어려움
- **모드 붕괴(Mode Collapse)**: Generator가 한 가지 이미지만 반복 생성
- **학습 불안정**: Generator와 Discriminator 간 학습 속도 균형이 어려움
- **평가 기준 부재**: 생성 이미지의 품질을 객관적으로 측정하기 어려움

### 시리즈 전체 요약

| 편 | 모델 | 핵심 |
|---|---|---|
| 1편 | Linear | 이미지 = 숫자, 학습 = weight 업데이트 |
| 2편 | FC NN | 계층을 쌓으면 파라미터가 폭발 |
| 3편 | CNN | 지역성 활용 → 적은 파라미터로 높은 성능 |
| 4편 | GAN | Generator vs Discriminator 경쟁 → 이미지 생성 |